In [1]:
import pandas as pd
import flair
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange,tqdm_notebook,tqdm

In [21]:
df = pd.read_excel('BNB_TWEETS.xlsx')
df.head()

,Unnamed: 0,Date,text,likeCount,retweetCount
0,0,2022-01-01 23:59:12+00:00,@TheMoonCarl @kasta_app @cz_binance Where can ...,1,0
1,1,2022-01-01 23:59:10+00:00,BTC diamond reversal and expected volatility f...,1,0
2,2,2022-01-01 23:59:10+00:00,@Dustin62025548 @binance @CEEK,1,0
3,3,2022-01-01 23:58:56+00:00,@MusadiqBabydoge @BabyDogeCoin @cz_binance @el...,1,0
4,4,2022-01-01 23:58:49+00:00,@cz_binance $KDA,3,1


In [22]:
df.shape

(570002, 5)

In [23]:
df['Date'] = pd.to_datetime(df['Date'])

In [24]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [28]:
df.dropna(inplace=True)

In [29]:
analyzer = SentimentIntensityAnalyzer()
compound=[]
pos=[]
neg=[]
neu=[]
for i,j in enumerate(tqdm(df['text'])):
    v = analyzer.polarity_scores(j)
    compound.append(v['compound'])
    pos.append(v['pos'])
    neg.append(v['neg'])
    neu.append(v['neu'])
df['compound'] = compound
df['pos'] = pos
df['neg'] = neg
df['neu'] = neu

100%|████████████████████████████████████████████████████████████████████████| 569997/569997 [02:11<00:00, 4321.25it/s]


In [ ]:
df['compound'] = df['compound'].round(decimals=1)
df['pos'] = df['pos'].round(decimals=1)
df['neg'] = df['neg'].round(decimals=1)
df['neu'] = df['neu'].round(decimals=1)

In [37]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'

2022-05-03 21:34:02,465 loading file C:\Users\nmahi\.flair\models\sentiment-en-mix-distillbert_4.pt


In [38]:
def sentiment_val_flair(flair_tag,flair_score):
    neg = 'NEGATIVE' in flair_tag
    val = float(flair_score)
    if neg:
        return -val
    return val

In [46]:
def get_sentiment_report(df):
    tb_sentiment_polarity_=[]
    tb_sentiment_subjectivity_=[]
    flair_sentiment_ = []
    for i,j in enumerate(tqdm(df['text'])):
        flair_s = flair.data.Sentence(j)
        flair_sentiment.predict(flair_s)
        flair_tag = flair_s.tag
        flair_score = flair_s.score
        flair_val = sentiment_val_flair(flair_tag,flair_score)
        flair_sentiment_.append(flair_val)
        tb_sentiment_polarity_.append(TextBlob(j).sentiment[0])
        tb_sentiment_subjectivity_.append(TextBlob(j).sentiment[1])
    df['Twitter_Flair'] = flair_sentiment_
    df['Twitter_TB_Polarity'] = tb_sentiment_polarity_
    df['Twitter_TB_Subjectivity'] = tb_sentiment_subjectivity_

In [47]:
get_sentiment_report(df)

100%|████████████████████████████████████████████████████████████████████████| 569997/569997 [9:30:25<00:00, 16.65it/s]


In [48]:
df.to_csv('BNB_TWEET_PROCESSED.csv')